# Feature Extraction
* **Products used:** 
[dem_cop_30](https://explorer.digitalearth.africa/products/s2_l2a), [s2_l2a](https://explorer.digitalearth.africa/products/dem_cop_90), [dem_srtm](https://explorer.digitalearth.africa/products/dem_srtm), [dem_srtm_deriv](https://explorer.digitalearth.africa/products/dem_srtm_deriv)

## Background:

Training data extraction plays a crucial role in training machine learning models. The process involves extracting relevant feature layers from a geospatial dataset based on predefined geometries or regions of interest. This enables the creation of accurate and reliable classification models for various applications such as land cover mapping, crop monitoring, and environmental analysis.

To facilitate this task, the open-data-cube provides a powerful function called "collect_training_data." This function is part of the deafrica_tools.classification script and is specifically designed to extract training data from the open-data-cube using geometries defined within a GeoJSON file. The GeoJSON file contains the spatial boundaries or polygons that delineate the regions of interest for which training data needs to be extracted.

## Description:

This notebook focuses on the extraction of training data (feature layers) from the open-data-cube using geometries defined within a GeoJSON file. It follows a step-by-step approach to guide users in utilizing the "collect_training_data" function effectively. The goal is to enable users to extract the appropriate training data for their specific use case.

The main steps in this notebook are as follows:

1. **Previewing the Training Data:** The notebook starts by plotting the polygons from the training data on a basemap. This visualization provides users with a visual representation of the regions of interest for which training data will be extracted.

2. **Defining the Feature Layer Function:** Next, a feature layer function is defined. This function specifies the set of feature layers to be extracted from the open-data-cube. These layers are carefully selected based on their relevance to the classification task at hand.

3. **Extracting Training Data:** The "collect_training_data" function is then employed to extract the training data from the datacube. It utilizes the predefined geometries from the GeoJSON file and retrieves the corresponding feature layers. This step ensures that the extracted data aligns precisely with the defined regions of interest.

4. **Exporting Training Data:** Finally, the extracted training data is exported and saved to disk. This facilitates its subsequent use in other scripts or machine learning workflows for training classification models.

By following the steps outlined in this notebook, users can leverage the "collect_training_data" function to efficiently extract training data from the open-data-cube. 

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [1]:
%matplotlib inline

import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rasterio
import xarray as xr
import richdem as rd
import os
import rioxarray
import matplotlib.pyplot as plt
from odc.io.cgroups import get_cpu_quota
from odc.algo import xr_geomedian
from datacube.testutils.io import rio_slurp_xarray
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.classification import collect_training_data

## Analysis parameters
 * path: The path to the input vector file from which we will extract training data. A default geojson is provided.
 * field: This is the name of column in your shapefile attribute table that contains the class labels. The class labels must be integers

In [2]:
# Specify a prefix to identify the area of interest in the saved outputs
# By assigning the desired prefix, you can easily identify the outputs associated with the specific area of interest.
prefix = 'Rwanda'

field = 'class_id'
path = f'data/{prefix}_training_samples.geojson'

print(path)

# Load input data shapefile
training_points= gpd.read_file(path) 
training_points.head()

data/Rwanda_training_samples.geojson


,class_id,class_name,geometry
0,0,Non-wetland,POINT (2966936.696 -269872.041)
1,0,Non-wetland,POINT (2879937.617 -279921.920)
2,0,Non-wetland,POINT (2844609.325 -285295.234)
3,0,Non-wetland,POINT (2849939.281 -338492.092)
4,0,Non-wetland,POINT (2884975.377 -221000.107)


In [3]:
# Set a flag to convert to polygons:
use_polygons = True

if use_polygons:
    # Convert from lat,lon to EPSG:6933 (projection in metres)
    training_points = training_points.to_crs("EPSG:6933")

    # Buffer geometry to get a square - only if trying to sample multiple pixels
    buffer_radius_m = 15
    training_points.geometry = training_points.geometry.buffer(buffer_radius_m, cap_style=3)

#### Plot on interactive map 

In [4]:
points = training_points
training_points.explore(
    tiles = "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}", 
    attr ='Imagery @2022 Landsat/Copernicus, Map data @2022 Google',
    popup=True,
    cmap='viridis',
    style_kwds=dict(radius= 5, color= 'red', fillOpacity= 0.8, fillColor= 'red', weight= 3),
    )

## Defining Query

The function `collect_training_data` takes our geojson containing class labels and extracts training data (features) from the datacube over the locations specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing any `NaN` or `inf` values.

The below variables can be set within the `collect_training_data` function:

* `field`: The name of column in your geojson file attribute table that contains the class labels, which corresponds to the `class_attr` that we defined earlier.
* `zonal_stats`: An optional string giving the names of zonal statistics to calculate across each geometry (polygon or point). Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', and 'min'.
* `dc_query`: A datacube query dictionary for the Open Data Cube query such as `measurements` (the bands to load from the satellite), the `resolution` (the cell size), and the `output_crs` (the output projection). 
* `feature_func`:  A function for generating feature layers that is applied to the data within the bounds of the input geometry. This function will take the 'dc_query' as the only argument.
* `return_coords`: If True, then the training data will contain two extra columns ‘x_coord’ and ‘y_coord’ corresponding to the x,y coordinate of each sample.

> Note: `collect_training_data` also has a number of additional parameters for handling ODC I/O read failures, where polygons that return an excessive number of null values can be resubmitted to the multiprocessing queue.  Check out the [docs](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/83116e80ebb4f8744e3de74e7a713aadd0a7577a/Tools/deafrica_tools/classification.py#L565) to learn more.

We will define the first three parameters and describe the `feature_func` seperately in a moment.

In [5]:
#set up our inputs to collect_training_data
zonal_stats = 'mean'

# Set up the inputs for the ODC query
time = ('2021')

# using nine spectral bands with 10~20 m spatial resolution
# measurements = ['blue','green','red','red_edge_1','red_edge_2', 'red_edge_3','nir_1','nir_2','swir_1','swir_2','emad','smad','bcmad']

resolution = (-20,20)

output_crs='epsg:6933'

Note that we've selected nine spectral bands with spatial resolution no lower than 20 m here for demonstration. However, it is advised that you test and select the bands based on your own classification task. Using the variables above, we can generate a datacube query object from the parameters above:

In [6]:
query = {
    'time': time,
    'output_crs': output_crs,
    'resolution': resolution,
}


### Defining feature function

## Defining feature function

To create the desired feature layers, we pass instructions to `collect_training_data` through the `feature_func` parameter. The `feature_func` must accept a `dc_query` dictionary, and return a single `xarray.Dataset` or `xarray.DataArray` containing 2D coordinates (i.e x, y - no time dimension). e.g.

          def feature_function(query):
              dc = datacube.Datacube(app='feature_layers')
              ds = dc.load(**query)
              ds = ds.mean('time')
              return ds

Below, we will define a more complicated feature layer function than the brief example shown above. Firstly We will calculate the Normalised Difference Water Index (NDWI), which is commonly used to distinguish between Water and non-water land cover classes. We use the `calculate_indices`function to automatically calculate NDVI for all specified bands. 

In addition, we'll use temporal signatures to help distinguish wetland classes. To reduce data size while keeping seasonal changes, we are implementing biannual temporal aggregation, i.e. geomedian (sometimes referred to as the 'geometric median') for each pixel location.

In [7]:
import s3fs

# set AWS region to access Sentinel-1 Annual mosaics 
s3 = s3fs.S3FileSystem(anon=True, client_kwargs={'region_name':'eu-central-1'})

s3.ls('esa-worldcover-s1/vvvhratio/')

['esa-worldcover-s1/vvvhratio/2020', 'esa-worldcover-s1/vvvhratio/2021']

In [8]:
def feature_layers(query):
    # connect to the datacube
    dc = datacube.Datacube(app='feature_layers')
    
    # load s2 annual geomedian
    ds = dc.load(
        product='gm_s2_annual',
        measurements = ['blue','green','red','red_edge_1','red_edge_2', 'red_edge_3','nir_1','nir_2','swir_1','swir_2','emad','smad','bcmad'],
        **query)
    
    # calculate some band indices
    ds = calculate_indices(ds,
                           index=['NDVI', 'MNDWI','TCW'],
                           drop=False,
                           satellite_mission='s2')
    
    
    # Add a prefix "Annual" to the band names
    new_band_names = ['Annual_' + band_name for band_name in ds.data_vars]
    ds = ds.rename({old_band_name: new_band_name for old_band_name, new_band_name in zip(ds.data_vars, new_band_names)})

    # stack multi-temporal measurements and rename them
    n_time = ds.dims['time']
    list_measurements = list(ds.keys())
    list_stack_measures = []
    for j in range(len(list_measurements)):
        for k in range(n_time):
            variable_name = list_measurements[j]+'_'+str(k)
            measure_single = ds[list_measurements[j]].isel(time=k).rename(variable_name)
            list_stack_measures.append(measure_single)
    ds_stacked = xr.merge(list_stack_measures, compat='override')

    
    # Load the Sentinel-1 data    
    ds_s1 = dc.load(product=["s1_rtc"],
                  measurements=['vv', 'vh'],
                  group_by="solar_day",
                  **query
                 )
    # Add a prefix "Sent1_" to the variables in ds_s1
    ds_s1 = ds_s1.rename({old_var: 'sent1_' + old_var for old_var in ds_s1.data_vars})

    # median values are used to scale the measurements so they have a similar range for visualization
    med_s1 = ds_s1[['sent1_vv','sent1_vh']].median()
    
    # Add ALOS L-Band Annual mosaic
    ds_alos = dc.load(product='alos_palsar_mosaic',
                      measurements=['hh','hv'],
                      **query)
    
    # Add a prefix "Sent1_" to the variables in ds_s1
    ds_alos = ds_alos.rename({old_var: 'alos_' + old_var for old_var in ds_alos.data_vars})
        
    med_alos = ds_alos[['alos_hh','alos_hv']].median()
    

    # Add WOfS Annual summary
    wofs_annual = dc.load(product='wofs_ls_summary_annual',
               like=ds.geobox,
               time=query['time'])
    wofs_annual_frequency = wofs_annual.frequency
    wofs_annual_frequency.name = 'WOfS_annual_frequency'
    
    
    # loop through the terrain attribite files and add them to the dataset
    folder = os.path.join("data/terrain_attributes/", prefix)
    for filename in os.listdir(folder):
            if filename.endswith('.tif'):
                filepath = os.path.join(folder, filename)
                tif = rio_slurp_xarray(filepath, gbox=ds.geobox)
                tif = tif.to_dataset(name=filename.replace('.tif', ''))
                ds_stacked = xr.merge([ds_stacked, tif], compat='override')


    # merge all the datasets into a single dataset
    ds_stacked = xr.merge([ds_stacked, med_s1, med_alos, wofs_annual_frequency], compat='override')

    return ds_stacked

Now let's run the `collect_training_data` function. This may take minutes to hours depending on your number of training points, number of measurements/bands set for the query and the calculation work in the feature function. Since we've used 10 measurements (9 spectral bands and 1 NDWI index) with 6 temporal geomedian for each band, it can be very time-consuming to finish the training features extraction. Therefore, here we are passing in `gdf=training_points[0:10]` to only run the code over the first 10 geometries as demonstration. Nevertheless, the extracted full training data file is provided in the 'Results/' folder, which will be used for next module of the workflow.

> **Note**:  With supervised classification, its common to have many, many labelled geometries in the training data. `collect_training_data` can parallelize across the geometries in order to speed up the extracting of training data. Setting `ncpus>1` will automatically trigger the parallelization. However, its best to set `ncpus=1` to begin with to assist with debugging before triggering the parallelization.

In [9]:
# detect the number of CPUs
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))


# collect training data
column_names, model_input = collect_training_data(
    gdf=training_points,
    dc_query=query,
    ncpus=ncpus,
    field=field, # integer class label
    zonal_stats=zonal_stats,
    feature_func=feature_layers,
    return_coords=True)

ncpus = 15
Taking zonal statistic: mean


  0%|          | 0/1628 [00:00<?, ?it/s]

Percentage of possible fails after run 1 = 0.0 %
Removed 490 rows wth NaNs &/or Infs
Output shape:  (1138, 51)


In [10]:
print(column_names)

['class_id', 'Annual_blue_0', 'Annual_green_0', 'Annual_red_0', 'Annual_red_edge_1_0', 'Annual_red_edge_2_0', 'Annual_red_edge_3_0', 'Annual_nir_1_0', 'Annual_nir_2_0', 'Annual_swir_1_0', 'Annual_swir_2_0', 'Annual_emad_0', 'Annual_smad_0', 'Annual_bcmad_0', 'Annual_NDVI_0', 'Annual_MNDWI_0', 'Annual_TCW_0', 'profile_curvature_30', 'tpi_100', 'DTW_FIA_4_ha', 'elevation', 'tpi_500', 'curvature_500', 'tpi_30', 'aspect_500', 'aspect_30', 'aspect_100', 'profile_curvature_500', 'slope_100', 'curvature_100', 'slope_30', 'mrrtf', 'twi_30', 'planform_curvature_30', 'mrvbf', 'curvature_30', 'planform_curvature_100', 'twi_500', 'planform_curvature_500', 'slope_500', 'DTW_FIA_0.25_ha', 'DTW_FIA_1_ha', 'profile_curvature_100', 'twi_100', 'sent1_vv', 'sent1_vh', 'alos_hh', 'alos_hv', 'WOfS_annual_frequency', 'x_coord', 'y_coord']


### Export training features

In [11]:
# convert the data to geopandas dataframe
pd_training_features=pd.DataFrame(data=model_input,columns=column_names)
#set the name and location of the output file
# output_file = "results/training_features.txt"
output_file = f"results/{prefix}_training_features.txt"
#Export files to disk
pd_training_features.to_csv(output_file, header=True, index=None, sep=' ')

In [12]:
# create geopandas dataframe
gpd_training_features=gpd.GeoDataFrame(pd_training_features, 
geometry=gpd.points_from_xy(model_input[:,-2], model_input[:,-1],crs=output_crs))

#####  Add a column for binary (wetland/non-wetland) classification

In [13]:
# Check if unique values in 'class_id' are only 0 and 1
unique_values = gpd_training_features['class_id'].unique()
if len(unique_values) == 2 and set(unique_values) == {0, 1}:
    # Replace 'class_id' with 'class_id_binary'
    gpd_training_features.rename(columns={'class_id': 'class_id_binary'}, inplace=True)
else:
    # Create 'class_id_binary' column based on condition
    gpd_training_features['class_id_binary'] = gpd_training_features['class_id'].apply(lambda x: 1 if x != 0 else 0)
    gpd_training_features.rename(columns={'class_id': 'class_id_type'}, inplace=True)

# Insert the new column at the second position
gpd_training_features.insert(0, 'class_id_binary', gpd_training_features.pop('class_id_binary'))
print(gpd_training_features.columns)

Index(['class_id_binary', 'class_id_type', 'Annual_blue_0', 'Annual_green_0',
       'Annual_red_0', 'Annual_red_edge_1_0', 'Annual_red_edge_2_0',
       'Annual_red_edge_3_0', 'Annual_nir_1_0', 'Annual_nir_2_0',
       'Annual_swir_1_0', 'Annual_swir_2_0', 'Annual_emad_0', 'Annual_smad_0',
       'Annual_bcmad_0', 'Annual_NDVI_0', 'Annual_MNDWI_0', 'Annual_TCW_0',
       'profile_curvature_30', 'tpi_100', 'DTW_FIA_4_ha', 'elevation',
       'tpi_500', 'curvature_500', 'tpi_30', 'aspect_500', 'aspect_30',
       'aspect_100', 'profile_curvature_500', 'slope_100', 'curvature_100',
       'slope_30', 'mrrtf', 'twi_30', 'planform_curvature_30', 'mrvbf',
       'curvature_30', 'planform_curvature_100', 'twi_500',
       'planform_curvature_500', 'slope_500', 'DTW_FIA_0.25_ha',
       'DTW_FIA_1_ha', 'profile_curvature_100', 'twi_100', 'sent1_vv',
       'sent1_vh', 'alos_hh', 'alos_hv', 'WOfS_annual_frequency', 'x_coord',
       'y_coord', 'geometry'],
      dtype='object')


In [14]:
# # Replace non-zero values in the 'class_id' column with 1
# gpd_training_features['class_id_binary'] = gpd_training_features['class_id_type'].apply(lambda x: 1 if x != 0 else 0)
# # Insert the new column at the second position
# gpd_training_features.insert(1, 'class_id_binary', gpd_training_features.pop('class_id_binary'))
# gpd_training_features

In [15]:
# save as geojson file
# gpd_training_features.to_file('results/training_features.geojson', driver="GeoJSON")
geojson_file = f"results/{prefix}_training_features.geojson"
gpd_training_features.to_file(geojson_file, driver="GeoJSON")

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [16]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2024-04-16'